# <span><h1 style = "font-family: garamond; font-size: 40px; font-style: normal; letter-spcaing: 3px; background-color: #f6f5f5; color :#fe346e; border-radius: 100px 100px; text-align:center">RETRIEVAL EVALUATION</h1></span>

In [1]:
%%capture

!pip install weaviate-client sentence-transformers rank_bm25 numpy py_vncorenlp rouge_score bert_score vncorenlp evaluate

In [2]:
import weaviate
from weaviate.client import Client
from weaviate.classes.query import Filter
from sentence_transformers import SentenceTransformer
from rank_bm25 import BM25Okapi, BM25Plus
import numpy as np
from nltk.tokenize import word_tokenize
import pandas as pd
import os

2025-11-22 18:12:48.762805: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1763835168.954689      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1763835169.006649      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

In [3]:
%%capture

!sudo apt-get update
!sudo apt-get install openjdk-8-jdk -y
!java -version

In [4]:
import numpy as np
from nltk.tokenize import word_tokenize
import pandas as pd
import os

# Thiết lập cho pandas
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
pd.set_option("display.width", None)
pd.set_option("display.max_colwidth", None)

In [5]:
!mkdir vncorenlp

In [6]:
%%capture

import py_vncorenlp

py_vncorenlp.download_model(save_dir='/kaggle/working/vncorenlp',)
nlp = py_vncorenlp.VnCoreNLP(save_dir='/kaggle/working/vncorenlp')

--2025-11-22 18:13:34--  https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/VnCoreNLP-1.2.jar
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 27412703 (26M) [application/octet-stream]
Saving to: ‘VnCoreNLP-1.2.jar’

     0K .......... .......... .......... .......... ..........  0% 1.29M 20s
    50K .......... .......... .......... .......... ..........  0% 1.29M 20s
   100K .......... .......... .......... .......... ..........  0% 6.13M 15s
   150K .......... .......... .......... .......... ..........  0% 2.36M 14s
   200K .......... .......... .......... .......... ..........  0% 8.02M 12s
   250K .......... .......... .......... .......... ..........  1% 9.87M 10s
   300K .......... .......... .......... .......... ..........  1% 12.3M 9s
   3

2025-11-22 18:13:47 INFO  WordSegmenter:24 - Loading Word Segmentation model
2025-11-22 18:13:47 INFO  PosTagger:23 - Loading POS Tagging model
2025-11-22 18:13:51 INFO  NerRecognizer:34 - Loading NER model
2025-11-22 18:14:04 INFO  DependencyParser:32 - Loading Dependency Parsing model


In [7]:
from vncorenlp import VnCoreNLP
from collections import Counter
import math
from nltk.translate.bleu_score import SmoothingFunction
from rouge_score import rouge_scorer
from bert_score import score as bert_score
import torch
import random

In [8]:
def set_SEED():
    SEED = 42
    random.seed(SEED)
    np.random.seed(SEED)
    torch.manual_seed(SEED)
    torch.cuda.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)
    torch.backends.cudnn.enabled = False
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True
    os.environ["PYTHONHASHSEED"] = str(SEED)

set_SEED()

In [9]:
filename = "/kaggle/input/vietnamhistory-dataset/vietnamese-stopwords.txt"  # Điều chỉnh đường dẫn nếu cần

# Đọc stopwords từ file txt
with open(filename, 'r', encoding='utf-8') as f:
    list_stopwords = f.read().splitlines()

print(f"Số lượng stopwords đã tải: {len(list_stopwords)}")

Số lượng stopwords đã tải: 3884


In [10]:
class StopwordRemover:
    def __init__(self, stopwords):
        self.stopwords = set(stopwords)

    def remove_stopwords(self, text):
        pre_text = []
        words = text.split()
        for word in words:
            if word.lower() not in self.stopwords:
                pre_text.append(word)
        return ' '.join(pre_text)
        
class VNPreprocessor:
    def __init__(self, nlp):
        self.nlp = nlp

    def preprocess(self, text):
        segmented_sentences = self.nlp.word_segment(text)
        processed_text = ' '.join(segmented_sentences)
        return processed_text
        
preprocessor = VNPreprocessor(nlp)
stopword_remover = StopwordRemover(list_stopwords)

In [11]:
import pandas as pd

train_csv_filename = '/kaggle/input/vietnamhistory-dataset/train_data.csv' # <-- Thay đổi tên file
corpus_data = pd.read_csv(train_csv_filename)
corpus_data = corpus_data[['period', 'context']]
corpus_data = corpus_data.drop_duplicates()
corpus_data['processed_context'] = corpus_data['context'].str.lower().apply(preprocessor.preprocess)
print(f"Số lượng mẫu trong tập test: {len(corpus_data)}")

test_csv_filename = '/kaggle/input/vietnamhistory-dataset/test_data.csv'
test_data = pd.read_csv(test_csv_filename)
test_data['processed_context'] = test_data['context'].str.lower().apply(preprocessor.preprocess)
print(f"Số lượng mẫu trong tập test: {len(test_data)}")

Số lượng mẫu trong tập test: 3515
Số lượng mẫu trong tập test: 1608


In [12]:
corpus_data.head()

,period,context,processed_context
0,Giai đoạn 1951 - 1954,"Từ những ngày đầu của cuộc kháng chiến, Đảng và Chính phủ ban hành nhiều chính sách về ruộng đất nhằm mang lại những quyền lợi bước đầu cho nông dân. Tuy vậy, chính sách ruộng đất của Đảng và Chính phủ thực hiện trong những năm đầu kháng chiến chưa phải là cải cách ruộng đất triệt để, mà trước hết chỉ ""nhằm hạn chế sự bóc lột phong kiến"", tập trung giải quyết những vấn đề giảm tô, giảm tức, chia ruộng đất của thực dân Pháp và Việt gian cho nông dân nghèo, sử dụng hợp lý ruộng đất vắng chủ, ruộng đất bỏ hoang, chia công điền công thổ, phát động phong trào hiến điền, thực hiện chính sách thuế công bằng dân chủ.","từ những ngày đầu của cuộc kháng_chiến , đảng và chính_phủ ban_hành nhiều chính_sách về ruộng_đất nhằm mang lại những quyền_lợi bước_đầu cho nông_dân . tuy_vậy , chính_sách ruộng_đất của đảng và chính_phủ thực_hiện trong những năm đầu kháng_chiến chưa phải là cải_cách_ruộng_đất triệt_để , mà trước_hết chỉ "" nhằm hạn_chế sự bóc_lột phong_kiến "" , tập_trung giải_quyết những vấn_đề giảm tô , giảm tức , chia ruộng_đất của thực_dân pháp và việt_gian cho nông_dân nghèo , sử_dụng hợp_lý ruộng_đất vắng chủ , ruộng_đất bỏ_hoang , chia công_điền công_thổ , phát_động phong_trào hiến điền , thực_hiện chính_sách thuế công_bằng dân_chủ ."
1,Giai đoạn 1951 - 1954,Kết quả là số thu năm 1952 đã đảm bảo được 78% số chi và năm 1953 lần đầu tiên kể từ Cách mạng tháng Tám số thu chẳng những hoàn toàn đảm bảo số chi mà còn vượt số chi tới 16%.,kết_quả là số thu năm 1952 đã đảm_bảo được 78% số chi và năm 1953 lần đầu_tiên kể từ cách_mạng_tháng_tám số thu chẳng_những hoàn_toàn đảm_bảo số chi mà_còn vượt số chi tới 16% .
2,Giai đoạn 1951 - 1954,"Tại Nam Bộ: Trước nguy cơ suy sụp ngày càng gia tăng của thực dân Pháp, đế quốc Mỹ nhanh chóng tranh thủ thời cơ bắt đầu thực hiện âm mưu trực tiếp can thiệp sâu vào Đông Dương. Các chuyến tàu viện trợ quân sự của Mỹ cập bến Sài Gòn ngày càng nhiều. Đối với miền Đông Nam Bộ, nỗ lực hoạt động của địch nằm trong phạm vi thực hiện chủ trương, phối hợp tiến công bằng cả kinh tế, chính trị, quân sự nhằm đánh mạnh vào lực lượng du kích, chiến lược chiến tranh du kích của ta, cướp vét và tận dụng dự trữ, thực hiện chính sách ""lấy chiến tranh nuôi chiến tranh"" và bao vây cô lập khu 7 bằng cách ngăn chặn, kiểm soát chặt chẽ khu 8... Từ đầu năm 1951 trở đi, địch ráo riết thiết lập các điểm chốt chặn trên dọc hành lang Đức Hòa - Hóc Môn, kiểm soát chặt đường biển, đường sông Vàm Cỏ, đường số 10 và số 1, cắt đứt đường giao thông tiếp viện của ta từ miền Tây lên miền Đông; tăng cường lực lượng bảo vệ các đường giao thông chiến lược nối thông với Tây Nguyên (đường 14, 20), với cực Nam Trung Bộ (đường 1 và đường xe lửa).","tại nam_bộ : trước nguy_cơ suy_sụp ngày_càng gia_tăng của thực_dân pháp , đế_quốc mỹ nhanh_chóng tranh_thủ thời_cơ bắt_đầu thực_hiện âm_mưu trực_tiếp can_thiệp sâu vào đông dương . các chuyến tàu viện_trợ quân_sự của mỹ cập bến sài_gòn ngày_càng nhiều . đối_với miền đông nam_bộ , nỗ_lực hoạt_động của địch nằm trong phạm_vi thực_hiện chủ_trương , phối_hợp tiến_công bằng cả kinh_tế , chính_trị , quân_sự nhằm đánh mạnh vào lực_lượng du_kích , chiến_lược chiến_tranh_du_kích của ta , cướp vét và tận_dụng dự_trữ , thực_hiện chính_sách "" lấy chiến_tranh nuôi chiến_tranh "" và bao_vây cô_lập khu 7 bằng cách ngăn_chặn , kiểm_soát chặt_chẽ khu 8 ... từ đầu năm 1951 trở đi , địch ráo_riết thiết_lập các điểm chốt chặn trên dọc hành_lang đức_hoà - hóc_môn , kiểm_soát chặt đường_biển , đường_sông vàm cỏ , đường số 10 và số 1 , cắt đứt đường giao_thông tiếp_viện của ta từ miền tây lên miền đông ; tăng_cường lực_lượng bảo_vệ các đường giao_thông chiến_lược nối thông với tây_nguyên ( đường 14 , 20 ) , với cực nam_trung_bộ ( đường 1 và đường xe_lửa ) ."
3,Giai đoạn 1945 - 1950,"Ở một số địa phương, bệnh hoa liễu phát triển mạnh do vệ sinh kém, tệ nạn gái mại dâm khá phổ biến. Ngoài ra là các bệnh sốt rét, tả, thương hàn cũng thường xảy ra, đặc biệt là 

In [13]:
import weaviate
from weaviate import AuthApiKey
from weaviate.classes.query import Filter

# --- WEAVIATE CONFIGURATION ---
WEAVIATE_URL = "v1h18i2sce1cl42acvvpa.c0.asia-southeast1.gcp.weaviate.cloud"
WEAVIATE_CLASS_NAME = "History"
WEAVIATE_API_KEY = "bTNhaG8xcHVlcjVtQXNMbV9UYmRKZEtFMzdTanhFWjlhUFBzWnVzTDl6WHFOUmY1REtkRHZQby9BNDB3PV92MjAw" 

client = None # Khởi tạo biến trước khi vào khối try
# Khởi tạo Weaviate Client (Sử dụng Client v4)
try:
    client = weaviate.connect_to_weaviate_cloud(
        cluster_url=WEAVIATE_URL,
        auth_credentials=AuthApiKey(WEAVIATE_API_KEY)
    )
    print("Kết nối Weaviate thành công:", client.is_live())
except Exception as e:
    print(f"Lỗi kết nối Weaviate: {e}")
    
if client is None or not client.is_live():
    # Dừng chương trình nếu không kết nối được để tránh NameError
    raise RuntimeError("Không thể kết nối đến Weaviate hoặc Client không hoạt động. Vui lòng kiểm tra lại cấu hình.")

/usr/local/lib/python3.11/dist-packages/weaviate/__init__.py:143: DeprecationWarning: Dep010: Importing AuthApiKey from weaviate is deprecated. Import AuthApiKey from its module: weaviate.auth
  _Warnings.root_module_import(name, map_[name])


Kết nối Weaviate thành công: True


In [14]:
class WeaviateRetriever:
    def __init__(self, weaviate_client, class_name):
        self.client = weaviate_client
        self.class_name = class_name
        self.encoder_model = SentenceTransformer("/kaggle/input/bge-m3-lora-embedding-models/transformers/default/1")
        print("WeaviateRetriever đã khởi tạo với mô hình BGE-M3.")

    def query(self, question, document_filter=None, topk=10):
        question = question.lower()
        question = preprocessor.preprocess(question)

        # --- Tạo vector từ câu hỏi bằng BGE-M3 ---
        question_embedding = self.encoder_model.encode(question, convert_to_tensor=False)
        
        # --- Xây dựng bộ lọc (API v4) ---
        weaviate_filter = None
        if document_filter:
            if not isinstance(document_filter, list):
                document_filter = [document_filter] 
            
            weaviate_filter = Filter.by_property("period").contains_any(document_filter)
        
        # --- Thực hiện truy vấn (API v4 ĐÃ SỬA return_metadata) ---
        try:
            results = self.client.collections.get(self.class_name).query.near_vector(
                near_vector=question_embedding,
                limit=topk,
                filters=weaviate_filter,
                return_properties=["context", "period"], 
                # SỬA LỖI: Chỉ yêu cầu certainty, uuid được lấy qua obj.uuid
                return_metadata=["certainty"] 
            )
        except Exception as e:
            print(f"Lỗi truy vấn Weaviate: {e}")
            return {}
        
        # --- Chuyển đổi kết quả (API v4) ---
        doc_results = results.objects
        
        if not doc_results:
            return {}
            
        # Sử dụng obj.uuid để lấy định danh, certainty từ metadata
        return {
        obj.uuid: obj.metadata.certainty # <--- ĐÃ SỬA: {UUID: Certainty Score}
        for obj in doc_results 
        if obj.metadata and obj.metadata.certainty is not None
    }
        
# Khởi tạo WeaviateRetriever
weaviate_retriever = WeaviateRetriever(
    weaviate_client=client, 
    class_name=WEAVIATE_CLASS_NAME
)

/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` 

WeaviateRetriever đã khởi tạo với mô hình BGE-M3.


In [15]:
import joblib
import pickle

# --- CẤU HÌNH ĐƯỜNG DẪN ĐẾN CÁC FILE MÔ HÌNH ---
# THAY THẾ 'PATH_TO_NOTEBOOK_OUTPUT' bằng đường dẫn thực tế đến thư mục output của Notebook huấn luyện
MODEL_DIR = "/kaggle/input/cs431-classify-model/pytorch/default/1" 

try:
    # 1. Tải Mô hình SVM đã huấn luyện
    classifier_model = joblib.load(os.path.join(MODEL_DIR, "best_svm.pkl"))
    
    # 2. Tải TF-IDF Vectorizer
    tfidf_vectorizer = joblib.load(os.path.join(MODEL_DIR, "tfidf.pkl"))
    
    # 3. Tải Label Encoder
    label_encoder = joblib.load(os.path.join(MODEL_DIR, "label_encoder.pkl"))
    
    print("✅ Đã tải thành công Mô hình SVM, TF-IDF, và Label Encoder.")
    
except Exception as e:
    print(f"❌ Lỗi khi tải mô hình: {e}. Vui lòng kiểm tra lại đường dẫn MODEL_DIR.")
    # Đặt các biến về None nếu tải lỗi
    classifier_model, tfidf_vectorizer, label_encoder = None, None, None
    
def documents_classification(question, classifier_model, vectorizer, label_encoder, top_k=1):
    """
    Phân loại câu hỏi để xác định giai đoạn lịch sử (period) tương ứng.
    
    Args:
        question (str): Câu hỏi đầu vào của người dùng.
        classifier_model: Mô hình phân loại đã tải (best_svm).
        vectorizer: TF-IDF Vectorizer đã tải.
        label_encoder: Label Encoder đã tải.
        top_k (int): Số lượng nhãn (period) dự đoán hàng đầu muốn trả về.

    Returns:
        list[str] | None: Danh sách các period dự đoán, hoặc None nếu không có mô hình.
    """
    if classifier_model is None or vectorizer is None or label_encoder is None:
        # Trả về None nếu không tải được mô hình
        return None 
    
    # --- 1. TIỀN XỬ LÝ (GIỐNG NHƯ KHI HUẤN LUYỆN) ---
    global preprocessor, stopword_remover # Đảm bảo truy cập các đối tượng toàn cục
    
    processed = question.lower()
    processed = preprocessor.preprocess(processed)
    processed = stopword_remover.remove_stopwords(processed)

    # --- 2. VECTOR HÓA ---
    # Sử dụng .transform (KHÔNG phải fit_transform)
    encoded = vectorizer.transform([processed])
    
    # --- 3. DỰ ĐOÁN XÁC SUẤT ---
    proba = classifier_model.predict_proba(encoded)[0]
    
    # --- 4. LẤY TOP K VÀ GIẢI MÃ ---
    top_k_idx = np.argsort(proba)[-top_k:][::-1]
    labels = label_encoder.inverse_transform(top_k_idx)
    
    # Trả về danh sách các period dự đoán
    return labels.tolist()

✅ Đã tải thành công Mô hình SVM, TF-IDF, và Label Encoder.


In [16]:
# Kiểm tra 1: Câu hỏi về chiến tranh
question_1 = "Diễn biến chính của Chiến dịch Điện Biên Phủ trên không là gì?"
filter_1 = documents_classification(question_1, classifier_model, tfidf_vectorizer, label_encoder, top_k=1)
print(f"Phân loại cho Q1: {filter_1}") 
# Kết quả mong đợi: ['thời đại hồ chí minh']

# Kiểm tra 2: Câu hỏi về phong kiến
question_2 = "Vị vua nào của triều Lý đã dời đô về Thăng Long?"
filter_2 = documents_classification(question_2, classifier_model, tfidf_vectorizer, label_encoder, top_k=1)
print(f"Phân loại cho Q2: {filter_2}") 
# Kết quả mong đợi: ['thời đại phong kiến'] (hoặc tương tự)

Phân loại cho Q1: ['Giai đoạn 1951 - 1954']
Phân loại cho Q2: ['Giai đoạn 1954 -1965']


In [17]:
tokenized_corpus = [doc.split() for doc in corpus_data['processed_context'].tolist()]
bm25 = BM25Plus(tokenized_corpus, k1=1.2, b=0.65) #SOTA

print("BM25 model đã được khởi tạo.")

BM25 model đã được khởi tạo.


In [18]:
class BM25Retriever:
    def __init__(self, bm25_model, documents, ids, stopword_remover, df, limit=10):
        self.bm25 = bm25_model
        self.documents = documents
        self.ids = ids
        self.stopword_remover = stopword_remover
        self.df = df
        self.limit = limit

        # Tạo từ điển ánh xạ từ doc_id sang vị trí index
        self.id_to_pos = {doc_id: idx for idx, doc_id in enumerate(self.ids)}

    def query(self, segmented_question, document_filter=None, limit=None, debug=False):
        if limit is None:
            limit = self.limit

        # 1. TIỀN XỬ LÝ VÀ TOKENIZE QUERY (Giữ nguyên)
        segmented_question = segmented_question.lower()
        segmented_question = preprocessor.preprocess(segmented_question)
        processed_query = self.stopword_remover.remove_stopwords(segmented_question)
        tokenized_query = processed_query.split()
        
        # 2. TÍNH TOÀN BỘ ĐIỂM SỐ BM25 (chỉ 1 lần)
        # BM25 model đã được tính toán trên toàn bộ corpus
        scores = self.bm25.get_scores(tokenized_query)

        # 3. ÁP DỤNG BỘ LỌC SAU KHI TÍNH ĐIỂM
        if document_filter:
            # Lấy index (dạng str) của các tài liệu thỏa mãn bộ lọc
            filtered_ids_set = set(self.df[self.df['period'].isin(document_filter)].index.astype(str).tolist())
            
            # Khởi tạo danh sách ID và Scores mới
            # Lặp qua tất cả ID và điểm số, chỉ giữ lại những ID nằm trong filtered_ids_set
            temp_scores = {}
            for doc_id, pos in self.id_to_pos.items():
                if doc_id in filtered_ids_set:
                    temp_scores[doc_id] = scores[pos]
            
            # Sử dụng điểm số đã lọc cho bước sắp xếp
            final_scores = temp_scores
        else:
            # Nếu không có bộ lọc, chuyển đổi toàn bộ scores array sang dict {doc_id: score}
            final_scores = {self.ids[i]: scores[i] for i in range(len(self.ids))}


        # 4. SẮP XẾP VÀ TRẢ VỀ TOP K
        sorted_results = sorted(final_scores.items(), key=lambda item: item[1], reverse=True)
        
        # Trả về kết quả {doc_id: score} cho Hybrid Fusion
        results = {doc_id: score for doc_id, score in sorted_results[:limit]}
        return results

class HybridRetriever:
    def __init__(self, biencoder, bm25, topk=10, alpha=0.5, fusion_method='scaling', classification_topk=1):
        self.biencoder = biencoder
        self.bm25 = bm25
        self.raw_topk= 50
        self.topk = topk
        self.alpha = alpha
        # Hỗ trợ 'scaling' hoặc 'rrf'
        self.fusion_method = fusion_method 
        # Tham số k nhỏ cho RRF (thường là 60)
        self.rrf_k = 60
        self.classification_topk = classification_topk

    def scale_scores(self, scores_dict):
        """
        Scale the scores in the dictionary to the range [0, 1].

        Args:
            scores_dict (dict): Dictionary with scores to scale.

        Returns:
            dict: Dictionary with scaled scores.
        """
        values = list(scores_dict.values())
        if not values:
            return {}
        
        min_val = min(values)
        max_val = max(values)

        if max_val == min_val:
            return {k: 1.0 for k in scores_dict}
        else:
            return {k: (v - min_val) / (max_val - min_val) for k, v in scores_dict.items()}

    def query(self, question, classifier_model=None, limit=None, debug=False):
        if limit is None:
            limit = self.topk
            
        # 1. GỌI HÀM PHÂN LOẠI
        document_filter = document_filter = documents_classification(
            question, 
            classifier_model, 
            tfidf_vectorizer, 
            label_encoder,
            top_k=self.classification_topk
        )

        if debug:
            print(f"--- DEBUG: Bắt đầu Truy vấn Hybrid ---")
            print(f"  Câu hỏi: '{question[:50]}...'")
            print(f"  Bộ lọc (Periods): {document_filter if document_filter else 'KHÔNG ÁP DỤNG (None)'}")
            
                
        # Truy vấn
        bm25_results = self.bm25.query(question, document_filter=document_filter, limit=self.raw_topk)
        biencoder_results = self.biencoder.query(question, document_filter=document_filter, topk=self.raw_topk)

        if debug:
            print(f"  BM25 (k={self.raw_topk}) trả về: {len(bm25_results)} docs.")
            print(f"  Weaviate (k={self.raw_topk}) trả về: {len(biencoder_results)} docs.")
            print(f"---------------------------------------")
            
        all_ids = set(bm25_results.keys()).union(set(biencoder_results.keys()))
        fused_scores = {}

        if self.fusion_method == 'rrf':
            # --- RRF (Reciprocal Rank Fusion) ---
            
            # Xây dựng từ điển Rank: {doc_id: rank}
            def rank_docs(results):
                return {doc_id: rank + 1 for rank, doc_id in enumerate(results.keys())}
            
            bm25_ranks = rank_docs(bm25_results)
            biencoder_ranks = rank_docs(biencoder_results)

            for doc_id in all_ids:
                rrf_score = 0
                
                # Tính RRF cho BM25
                rank_bm25 = bm25_ranks.get(doc_id, self.rrf_k + 1)
                rrf_score += 1 / (self.rrf_k + rank_bm25)
                
                # Tính RRF cho Bi-encoder
                rank_biencoder = biencoder_ranks.get(doc_id, self.rrf_k + 1)
                rrf_score += 1 / (self.rrf_k + rank_biencoder)
                
                fused_scores[doc_id] = rrf_score
                
        elif self.fusion_method == 'scaling':
            # --- SCALING (Normalization Fusion) ---
            
            bm25_scaled = self.scale_scores(bm25_results)
            biencoder_scaled = self.scale_scores(biencoder_results)
            
            for doc_id in all_ids:
                bm25_score = bm25_scaled.get(doc_id, 0)
                biencoder_score = biencoder_scaled.get(doc_id, 0)

                # Sử dụng alpha đã định nghĩa trong __init__
                fused_score = ((1-self.alpha) * bm25_score) + (self.alpha * biencoder_score)
                fused_scores[doc_id] = fused_score
                
        else:
            raise ValueError(f"Fusion method '{self.fusion_method}' không hợp lệ.")

        # Sắp xếp và trả về top K
        fused_sorted = sorted(fused_scores.items(), key=lambda item: item[1], reverse=True)
        top_docs = [doc_id for doc_id, score in fused_sorted[:limit]]

        return top_docs

In [19]:
# Khởi tạo BM25Retriever
bm25_retriever = BM25Retriever(
    bm25_model=bm25,
    documents=corpus_data['processed_context'].tolist(),
    ids=corpus_data.index.astype(str).tolist(),
    stopword_remover=stopword_remover,
    df=corpus_data,
    limit=10
)

# Khởi tạo ChromaDBRetriever
weaviate_retriever = WeaviateRetriever(
    weaviate_client=client,
    class_name=WEAVIATE_CLASS_NAME
)

# Khởi tạo HybridRetriever
hybrid_retriever = HybridRetriever(
    biencoder=weaviate_retriever,
    bm25=bm25_retriever,
    topk=10
)
print("Các retriever đã được khởi tạo thành công!")


WeaviateRetriever đã khởi tạo với mô hình BGE-M3.
Các retriever đã được khởi tạo thành công!


In [20]:
corpus_data_sample = corpus_data.sample(2)
corpus_data_sample

,period,context,processed_context
4185,Giai đoạn 1965 -1975,"Chỉ trong một thời gian ngắn, đã có hàng triệu lá đơn tình nguyện đăng ký phong trào Ba sẵn sàng, Ba đảm đang, Năm xung phong của nam, nữ thanh niên. Chỉ riêng năm 1965 có 290.000 thanh niên nhập ngũ, riêng trong tháng 5 đã có 150.000 người. Những tháng cuối năm 1965, hàng chục nghìn thanh niên và quân nhân chuyển ngành hoặc phục viên được động viên vào quân đội, hàng chục nghìn người khác được gọi vào lực lượng thanh niên xung phong, dân công hỏa tuyến.","chỉ trong một thời_gian ngắn , đã có hàng triệu lá đơn tình_nguyện đăng_ký phong_trào ba sẵn_sàng , ba đảm_đang , năm xung_phong của nam , nữ thanh_niên . chỉ riêng năm 1965 có 290.000 thanh_niên nhập_ngũ , riêng trong tháng 5 đã có 150.000 người . những tháng cuối năm 1965 , hàng chục nghìn thanh_niên và quân_nhân chuyển ngành hoặc phục_viên được động_viên vào quân_đội , hàng chục nghìn người khác được gọi vào lực_lượng thanh_niên xung_phong , dân_công hoả_tuyến ."
72,Giai đoạn 1951 - 1954,"Đây là thuế đánh vào các ngành kinh doanh công nghiệp, tiểu công nghiệp và thương nghiệp, bao gồm các doanh nghiệp hoạt động thường xuyên, có cơ sở, tương đối khá, các quán hàng và hoạt động buôn chuyến. Thuế công, thương nghiệp được ban hành theo sắc lệnh số 44/SL ngày 22-7-1951. Mức thu thuế tương đối nhẹ so với thuế nông nghiệp, chỉ chiếm 15% thu nhập, nhằm khuyến khích phát triển công nghiệp, tiểu công nghiệp và lưu thông hàng hóa vì công, thương nghiệp của ta còn nhỏ bé, phân tán, chưa ổn định.","đây là thuế đánh vào các ngành kinh_doanh công_nghiệp , tiểu_công_nghiệp và thương_nghiệp , bao_gồm các doanh_nghiệp hoạt_động thường_xuyên , có cơ_sở , tương_đối khá , các quán hàng và hoạt_động buôn_chuyến . thuế công , thương_nghiệp được ban_hành theo sắc_lệnh số 44 / sl ngày 22-7-1951 . mức thu thuế tương_đối nhẹ so với thuế nông_nghiệp , chỉ chiếm 15% thu_nhập , nhằm khuyến_khích phát_triển công_nghiệp , tiểu_công_nghiệp và lưu_thông hàng_hoá vì công , thương_nghiệp của ta còn nhỏ_bé , phân_tán , chưa ổn_định ."


In [21]:
corpus_data.shape

(3515, 3)

In [22]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer, util
from tqdm.notebook import tqdm
# Giả định HybridRetriever, BM25Retriever, WeaviateRetriever đã được định nghĩa và import

def evaluate_rag_system(test_df, retriever, sbert_model, corpus_df, weaviate_uuid_to_context_map, topk_list):
    """
    Đánh giá hiệu suất truy xuất (Recall@K) của hệ thống RAG.
    
    Arguments:
        test_df (pd.DataFrame): Tập dữ liệu TEST (chứa câu hỏi và ground_truth context).
        retriever: HybridRetriever, BM25Retriever, hoặc WeaviateRetriever đã khởi tạo.
        sbert_model (str):  mô hình Sentence Transformer.
        corpus_df (pd.DataFrame): Tập dữ liệu TRAIN (Corpus) dùng để xây dựng BM25.
        weaviate_uuid_to_context_map (dict): Ánh xạ {UUID: Context} từ Weaviate (tập TRAIN).
        topk_list (list): Danh sách các giá trị K để đánh giá.
    """
    
    similarity_scores_dict = {}
    for k in topk_list:
        similarity_scores_dict[f'R@{k}'] = [] # Khóa cho Recall/Success
        similarity_scores_dict[f'P@{k}'] = [] # Khóa cho Precision
    
    SBERT_ENCODE_BATCH_SIZE = 16

    # Tạo từ điển ánh xạ ID (index) của BM25 sang Context (tập TRAIN)
    bm25_id_to_context = dict(zip(corpus_df.index.astype(str), corpus_df['processed_context']))

    # Dùng cho việc in kết quả (Tùy chọn)
    print(f"Bắt đầu đánh giá {retriever.__class__.__name__} trên {len(test_df)} mẫu TEST...")

    for idx, row in tqdm(test_df.iterrows()):
        question = row['question']
        ground_truth = row['processed_context'] # Context Gốc (từ TEST)
        limit = max(topk_list)
        
        # Chỉ debug mẫu đầu tiên (idx == 0)
        debug_mode = (idx == 0)
        retrieved_results = retriever.query(question, limit=limit, debug=debug_mode,classifier_model=classifier_model)
        
        # --- 1. Xử lý kết quả truy vấn và lấy ID ---
        filtered_doc_ids = []
        if isinstance(retriever, HybridRetriever):
            # HybridRetriever trả về list các tuple (doc_id, score) (tùy thuộc vào cách bạn sửa)
            # Nếu đã sửa để trả về list ID:
            filtered_doc_ids = retrieved_results 
        elif isinstance(retriever, BM25Retriever):
            filtered_doc_ids = list(retrieved_results.keys()) # BM25: keys là ID (index str)
        elif isinstance(retriever, WeaviateRetriever):
            # Weaviate: keys là UUID (str)
            filtered_doc_ids = list(retrieved_results.keys()) 
        else:
            continue

        # --- 2. Ánh xạ ID sang Context TẬP TRAIN (CORPUS) ---
        retrieved_contexts = []
        for doc_id in filtered_doc_ids:
            context = None
            
            # Ưu tiên tìm kiếm trong ánh xạ UUID (dành cho Weaviate)
            if doc_id in weaviate_uuid_to_context_map:
                context = weaviate_uuid_to_context_map.get(doc_id)
            
            # Nếu không phải UUID (có thể là index của BM25)
            elif doc_id in bm25_id_to_context:
                context = bm25_id_to_context.get(doc_id)

            if context:
                retrieved_contexts.append(context)


        combined_contexts = retrieved_contexts
        
        if not combined_contexts:
            # Xử lý trường hợp không có context nào được truy xuất
            # ... (Phần này không đổi) ...
            print(f"Không tìm thấy context nào cho câu hỏi: '{question}'")
            for k in topk_list:
                similarity_scores_dict[k].append(0)
            continue
            
        # --- 3. Tính Cosine Similarity ---
        # Ground Truth Embedding (từ TEST)
        ground_truth_embedding = sbert_model.encode(ground_truth, convert_to_tensor=True, batch_size=SBERT_ENCODE_BATCH_SIZE)
        # Retrieved Contexts Embeddings (từ TRAIN)
        retrieved_embeddings = sbert_model.encode(combined_contexts, convert_to_tensor=True, batch_size=SBERT_ENCODE_BATCH_SIZE)

        cosine_scores = util.cos_sim(ground_truth_embedding, retrieved_embeddings)[0].cpu().numpy()

        # Thêm lệnh giải phóng bộ nhớ
        del ground_truth_embedding
        del retrieved_embeddings
        torch.cuda.empty_cache()

        # --- 4. Tính điểm Recall@K và Precision@K ---
        for k in topk_list:
            topk_cos_scores = cosine_scores[:k]
            max_similarity = np.max(topk_cos_scores) if topk_cos_scores.size > 0 else 0
            
            # --- TÍNH RECALL@K (Success@K) ---
            # Trả lời câu hỏi: Có tìm thấy tài liệu liên quan cao nào trong Top K không?
            # Ngưỡng: Độ tương đồng >= 0.95
            recall_at_k = 1 if max_similarity >= 0.95 else 0
            similarity_scores_dict[f'R@{k}'].append(recall_at_k)

            # --- TÍNH PRECISION@K (Semantic Precision) ---
            # Trả lời câu hỏi: Tỷ lệ tài liệu liên quan (theo ngữ nghĩa) trong Top K là bao nhiêu?
            # Tài liệu được coi là liên quan nếu Similarity >= 0.95
            relevant_docs_in_k = np.sum(topk_cos_scores >= 0.95)
            precision_at_k = relevant_docs_in_k / k if k > 0 else 0
            similarity_scores_dict[f'P@{k}'].append(precision_at_k)

    # Tính điểm đánh giá trung bình
    average_scores = {k: np.mean(v) for k, v in similarity_scores_dict.items()}
    return test_df, average_scores

In [23]:
from weaviate.classes.query import MetadataQuery
# ... (Phần kết nối client) ...

history_collection = client.collections.get("History")

print("\n--- Bắt đầu truy vấn Weaviate bằng Cursor để tạo bộ ánh xạ UUID ---")

weaviate_uuid_to_context_map = {}
count = 0
limit_data_out = 10000 
batch_size = 100 # Kích thước lô để lấy dữ liệu

# Sử dụng iterator/cursor để lấy dữ liệu an toàn hơn fetch_objects
for obj in history_collection.iterator(
    include_vector=False, # Không cần vector
    return_properties=["context"], # Chỉ cần context
):
    
    # 1. Kiểm tra UUID (obj.uuid là vị trí khả năng nhất trong v4)
    retrieved_uuid = obj.uuid
    
    if retrieved_uuid:
        # 2. Lấy Context
        context = obj.properties.get('context')
        
        if context:
            weaviate_uuid_to_context_map[retrieved_uuid] = context
            count += 1
            if count >= limit_data_out:
                break # Dừng lại nếu đạt giới hạn

    else:
        # Nếu vẫn không lấy được UUID, đây là lỗi nghiêm trọng của Client API
        # Nhưng với iterator, obj.uuid thường có sẵn
        print("LỖI NGHIÊM TRỌNG: Iterator không trả về UUID. Vui lòng kiểm tra lại phiên bản Client Weaviate.")
        break
    
print(f"Đã tạo bộ ánh xạ với {len(weaviate_uuid_to_context_map)} Contexts.")


--- Bắt đầu truy vấn Weaviate bằng Cursor để tạo bộ ánh xạ UUID ---
Đã tạo bộ ánh xạ với 3515 Contexts.


In [24]:
def run_hybrid_experiments(
    test_df, sbert_model, corpus_df, weaviate_uuid_to_context_map, 
    weaviate_retriever, bm25_retriever, list_k
):
    """Thử nghiệm các giá trị K phân loại, alpha và phương pháp fusion khác nhau."""
    
    # 🌟 BƯỚC SỬA ĐỔI CHÍNH: TẠO DANH SÁCH TÊN CỘT MỚI
    new_list_k = []
    for k in list_k:
        new_list_k.append(f'R@{k}')
        new_list_k.append(f'P@{k}')
    
    experiment_results = []
    
    # 1. GIAI ĐOẠN 1: TÌM ALPHA TỐI ƯU VỚI BỘ LỌC RỘNG NHẤT (K_class=4)
    classification_k_list = [4] # CHỈ CHẠY K=4 LẦN ĐẦU
    
    for class_k in classification_k_list:
        print(f"\n======================================================================")
        print(f"--- BẮT ĐẦU THỬ NGHIỆM VỚI PHÂN LOẠI K={class_k} ---")
        print(f"======================================================================")
        
        # --- EXPERIMENT 1: Normalization Scaling với Alpha từ 0.0 đến 1.0 (bước 0.1) ---
        print("\n--- Thử nghiệm 1: Normalization Scaling (Alpha Tuning) ---")
        alpha_values = np.arange(0.0, 1.01, 0.1)
        
        for alpha in tqdm(alpha_values, desc=f"Tuning Alpha (K_class={class_k})"):
            current_alpha = round(alpha, 1)
            hybrid_retriever = HybridRetriever(
                biencoder=weaviate_retriever,
                bm25=bm25_retriever,
                alpha=current_alpha,
                fusion_method='scaling',
                topk=max(list_k),
                classification_topk=class_k # <--- TRUYỀN K PHÂN LOẠI
            )
            
            # Chạy đánh giá
            _, scores = evaluate_rag_system(
                test_df=test_df, 
                retriever=hybrid_retriever, 
                sbert_model=sbert_model, 
                corpus_df=corpus_df,
                weaviate_uuid_to_context_map=weaviate_uuid_to_context_map,
                topk_list=list_k
            )
            
            # Lưu kết quả
            result = {
                'Classification K': class_k, # <--- LƯU K PHÂN LOẠI
                'Fusion Method': 'Normalization Scaling',
                'Alpha': current_alpha,
                **scores # scores chứa R@K và P@K
            }
            experiment_results.append(result)

            # In kết quả tạm thời
            temp_df = pd.DataFrame([result])
            temp_df['Alpha'] = temp_df['Alpha'].fillna('N/A')
            
            # SỬ DỤNG new_list_k ĐỂ LÀM TRÒN VÀ CHỌN CỘT
            temp_df[new_list_k] = temp_df[new_list_k].apply(lambda x: round(x, 3)) # <--- SỬ DỤNG new_list_k
            temp_df_display = temp_df[['Classification K', 'Fusion Method', 'Alpha'] + new_list_k] # <--- SỬ DỤNG new_list_k
            
            print(f"\n✅ Kết quả Alpha={current_alpha} (Scaling):")
            print(temp_df_display.to_markdown(index=False))
            # ----------------------------------------------------------------------

        # --- EXPERIMENT 2: Reciprocal Rank Fusion (RRF) ---
        print("\n--- Thử nghiệm 2: Reciprocal Rank Fusion (RRF) ---")
        
        # RRF không cần Alpha
        hybrid_rrf = HybridRetriever(
            biencoder=weaviate_retriever,
            bm25=bm25_retriever,
            fusion_method='rrf',
            topk=max(list_k),
            classification_topk=class_k # <--- TRUYỀN K PHÂN LOẠI
        )
        
        _, scores_rrf = evaluate_rag_system(
            test_df=test_df, 
            retriever=hybrid_rrf, 
            sbert_model=sbert_model, 
            corpus_df=corpus_df,
            weaviate_uuid_to_context_map=weaviate_uuid_to_context_map,
            topk_list=list_k
        )
        
        result_rrf = {
            'Classification K': class_k, # <--- LƯU K PHÂN LOẠI
            'Fusion Method': 'RRF',
            'Alpha': None,
            **scores_rrf
        }
        experiment_results.append(result_rrf)
        
        # 2. IN KẾT QUẢ RRF NGAY LẬP TỨC
        temp_df_rrf = pd.DataFrame([result_rrf])
        temp_df_rrf['Alpha'] = temp_df_rrf['Alpha'].fillna('N/A')
        
        temp_df_rrf[new_list_k] = temp_df_rrf[new_list_k].apply(lambda x: round(x, 3))
        temp_df_rrf_display = temp_df_rrf[['Classification K', 'Fusion Method', 'Alpha'] + new_list_k]
        
        print("\n✅ Kết quả RRF:")
        print(temp_df_rrf_display.to_markdown(index=False))
        # ----------------------------------------------------------------------
    
    # --- 3. IN KẾT QUẢ TỔNG HỢP CUỐI CÙNG ---
    results_df = pd.DataFrame(experiment_results)
    
    # Xử lý kết quả tổng hợp: FILLNA('N/A') và làm tròn
    results_df['Alpha'] = results_df['Alpha'].fillna('N/A')
    results_df[new_list_k] = results_df[new_list_k].apply(lambda x: round(x, 3)) # <--- SỬ DỤNG new_list_k
    
    return results_df

In [25]:
# test_data = test_data[:30]

In [26]:
# THAY THẾ ĐƯỜNG DẪN MÔ HÌNH NHÚNG
sbert_model_path = "/kaggle/input/bge-m3-lora-embedding-models/transformers/default/1"
sbert_model = SentenceTransformer(sbert_model_path)
list_k = [1, 2, 3, 4, 5, 10, 20, 30]

# # Đánh giá hệ thống trên tập test với HybridRetriever
# df_test, average_similarity = evaluate_rag_system(
#     test_df=test_data,
#     retriever=hybrid_retriever, # Đánh giá Hybrid
#     sbert_model=sbert_model,
#     corpus_df=corpus_data,
#     weaviate_uuid_to_context_map=weaviate_uuid_to_context_map,
#     topk_list = list_k
# )

# # In kết quả
# print("Average Scores (Hybrid Retriever):", average_similarity)

# --- LỆNH GỌI THỬ NGHIỆM ---
# Lưu ý: Cần đảm bảo tất cả các biến như 'test_data', 'corpus_data', 
# 'weaviate_retriever', 'bm25_retriever', v.v. đã được định nghĩa.

results_df = run_hybrid_experiments(
    test_df=test_data, 
    sbert_model=sbert_model, 
    corpus_df=corpus_data, 
    weaviate_uuid_to_context_map=weaviate_uuid_to_context_map,
    weaviate_retriever=weaviate_retriever, 
    bm25_retriever=bm25_retriever, 
    list_k=list_k
)

print("\n✅ Kết quả tổng hợp:")
print(results_df)


--- BẮT ĐẦU THỬ NGHIỆM VỚI PHÂN LOẠI K=4 ---

--- Thử nghiệm 1: Normalization Scaling (Alpha Tuning) ---


Tuning Alpha (K_class=4):   0%|          | 0/11 [00:00<?, ?it/s]

Bắt đầu đánh giá HybridRetriever trên 1608 mẫu TEST...


0it [00:00, ?it/s]

--- DEBUG: Bắt đầu Truy vấn Hybrid ---
  Câu hỏi: 'Nhân dân Việt Nam đã đánh thắng quân đội Pháp dựa ...'
  Bộ lọc (Periods): ['Giai đoạn 1945 - 1950', 'Giai đoạn 1951 - 1954', 'Giai đoạn 1954 -1965', 'Giai đoạn 1965 -1975']
  BM25 (k=50) trả về: 50 docs.
  Weaviate (k=50) trả về: 50 docs.
---------------------------------------

✅ Kết quả Alpha=0.0 (Scaling):
|   Classification K | Fusion Method         |   Alpha |   R@1 |   P@1 |   R@2 |   P@2 |   R@3 |   P@3 |   R@4 |   P@4 |   R@5 |   P@5 |   R@10 |   P@10 |   R@20 |   P@20 |   R@30 |   P@30 |
|-------------------:|:----------------------|--------:|------:|------:|------:|------:|------:|------:|------:|------:|------:|------:|-------:|-------:|-------:|-------:|-------:|-------:|
|                  4 | Normalization Scaling |       0 | 0.558 | 0.558 |  0.64 | 0.322 | 0.674 | 0.227 | 0.692 | 0.174 | 0.705 | 0.142 |  0.752 |  0.076 |  0.779 |  0.039 |  0.789 |  0.027 |
Bắt đầu đánh giá HybridRetriever trên 1608 mẫu TEST...


...

--- DEBUG: Bắt đầu Truy vấn Hybrid ---
  Câu hỏi: 'Nhân dân Việt Nam đã đánh thắng quân đội Pháp dựa ...'
  Bộ lọc (Periods): ['Giai đoạn 1945 - 1950', 'Giai đoạn 1951 - 1954', 'Giai đoạn 1954 -1965', 'Giai đoạn 1965 -1975']
  BM25 (k=50) trả về: 50 docs.
  Weaviate (k=50) trả về: 50 docs.
---------------------------------------

✅ Kết quả Alpha=0.9 (Scaling):
|   Classification K | Fusion Method         |   Alpha |   R@1 |   P@1 |   R@2 |   P@2 |   R@3 |   P@3 |   R@4 |   P@4 |   R@5 |   P@5 |   R@10 |   P@10 |   R@20 |   P@20 |   R@30 |   P@30 |
|-------------------:|:----------------------|--------:|------:|------:|------:|------:|------:|------:|------:|------:|------:|------:|-------:|-------:|-------:|-------:|-------:|-------:|
|                  4 | Normalization Scaling |     0.9 | 0.502 | 0.502 | 0.591 | 0.297 | 0.627 |  0.21 | 0.652 | 0.164 | 0.669 | 0.135 |   0.71 |  0.072 |  0.742 |  0.042 |  0.771 |  0.039 |
Bắt đầu đánh giá HybridRetriever trên 1608 mẫu TEST...


0it [00:00, ?it/s]

--- DEBUG: Bắt đầu Truy vấn Hybrid ---
  Câu hỏi: 'Nhân dân Việt Nam đã đánh thắng quân đội Pháp dựa ...'
  Bộ lọc (Periods): ['Giai đoạn 1945 - 1950', 'Giai đoạn 1951 - 1954', 'Giai đoạn 1954 -1965', 'Giai đoạn 1965 -1975']
  BM25 (k=50) trả về: 50 docs.
  Weaviate (k=50) trả về: 50 docs.
---------------------------------------

✅ Kết quả Alpha=1.0 (Scaling):
|   Classification K | Fusion Method         |   Alpha |   R@1 |   P@1 |   R@2 |   P@2 |   R@3 |   P@3 |   R@4 |   P@4 |   R@5 |   P@5 |   R@10 |   P@10 |   R@20 |   P@20 |   R@30 |   P@30 |
|-------------------:|:----------------------|--------:|------:|------:|------:|------:|------:|------:|------:|------:|------:|------:|-------:|-------:|-------:|-------:|-------:|-------:|
|                  4 | Normalization Scaling |       1 | 0.502 | 0.502 | 0.591 | 0.297 | 0.627 |  0.21 | 0.652 | 0.164 | 0.669 | 0.135 |   0.71 |  0.072 |  0.742 |  0.037 |  0.767 |  0.026 |

--- Thử nghiệm 2: Reciprocal Rank Fusion (RRF) ---
Bắt đầu đánh

0it [00:00, ?it/s]

--- DEBUG: Bắt đầu Truy vấn Hybrid ---
  Câu hỏi: 'Nhân dân Việt Nam đã đánh thắng quân đội Pháp dựa ...'
  Bộ lọc (Periods): ['Giai đoạn 1945 - 1950', 'Giai đoạn 1951 - 1954', 'Giai đoạn 1954 -1965', 'Giai đoạn 1965 -1975']
  BM25 (k=50) trả về: 50 docs.
  Weaviate (k=50) trả về: 50 docs.
---------------------------------------

✅ Kết quả RRF:
|   Classification K | Fusion Method   | Alpha   |   R@1 |   P@1 |   R@2 |   P@2 |   R@3 |   P@3 |   R@4 |   P@4 |   R@5 |   P@5 |   R@10 |   P@10 |   R@20 |   P@20 |   R@30 |   P@30 |
|-------------------:|:----------------|:--------|------:|------:|------:|------:|------:|------:|------:|------:|------:|------:|-------:|-------:|-------:|-------:|-------:|-------:|
|                  4 | RRF             | N/A     | 0.532 | 0.532 | 0.647 |  0.53 |  0.69 | 0.383 | 0.715 |  0.31 | 0.727 | 0.256 |   0.76 |  0.139 |   0.79 |  0.074 |  0.799 |   0.05 |

✅ Kết quả tổng hợp:
    Classification K          Fusion Method Alpha    R@1    P@1    R@2    P@2